In [1]:
import os, time, copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from PIL import Image

from utils import *
from dataset import *
from metrics import *
from FCN.fcn32_alex import *

In [2]:
set_seed(1)
os.environ["CUDA_VISIBLE_DEVICES"] = '7'


In [3]:
# Load data
train_data = VOC(mode = 'train', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
train_loader = data.DataLoader(train_data, batch_size=1, num_workers = 0, shuffle=True)

val_data = VOC(mode = 'val', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
val_loader = data.DataLoader(val_data, batch_size=1, num_workers = 0, shuffle=True)

In [4]:
def train(model, device, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0

    loss_sum = torch.zeros(1)
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    optimizer.zero_grad()
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
            
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc += pixel_accuracy(out_mask, target_mask)/20
        macc += mean_accuracy(out_mask, target_mask)/20
        mIU += mean_IU(out_mask, target_mask)/20
        fIU += frequency_weighted_IU(out_mask, target_mask)/20
        
        #calculate loss
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)/20
        loss.backward()
        loss_sum += loss
        
        
        # update gradient every 20 image
        if i%20 == 19 or i == 1111:
            optimizer.step()
            print("Batch %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i//20, loss_sum, pacc, macc, mIU, fIU))
            
            running_loss += loss_sum.item() * 20
            r_pacc += pacc * 20
            r_macc += macc * 20
            r_mIU += mIU * 20
            r_fIU += fIU * 20
            
            # batch initialize
            loss_sum, pacc, macc, mIU, fIU = 0, 0, 0, 0, 0
            optimizer.zero_grad()
            
            # show output mask
#             img = decode_segmap(out_mask)
#             plt.imshow(img); plt.axis('off'); plt.show()
        
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1112 for m in epoch_metric]
    
    print('Train:\t%s' %epoch_metric)
    return epoch_metric

In [5]:
def validate(model, device, val_loader, criterion, mode = 'val'):
    # switch to evaluate mode
    model.eval()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc = pixel_accuracy(out_mask, target_mask)
        macc = mean_accuracy(out_mask, target_mask)
        mIU = mean_IU(out_mask, target_mask)
        fIU = frequency_weighted_IU(out_mask, target_mask)
        
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)
        
        if i % 100 == 99:
            print("Val %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i, loss, pacc, macc, mIU, fIU))
        # statistics
        running_loss += loss.item()
        r_pacc += pacc
        r_macc += macc
        r_mIU += mIU
        r_fIU += fIU

    #epoch_loss = running_loss / 1111
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1111 for m in epoch_metric]
    
    if mode == 'val':
        print('Val metric:\t%s' %epoch_metric)
    else:
        print('Test metric:\t%s' %epoch_metric)
    
    return epoch_metric  #, copy.deepcopy(model.state_dict())

In [10]:
def train_model(model, device, train_loader, val_loader, lr = 1e-4, dict_name = 'fcn32', num_epochs=50):
    
    criterion = nn.CrossEntropyLoss(ignore_index = 255)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5**(-4))

    if device == 'cuda':
        model.to(device)
        cudnn.benchmark = True
        criterion = criterion.cuda()
    
    since = time.time()
    M = {'train' : [], 'val' : []}
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        
        train_metric = train(model, device, train_loader, criterion, optimizer)
        M['train'].append(train_metric)
        
        # evaluate on validation set
        val_metric = validate(model, device, val_loader, criterion)
        M['val'].append(val_metric)
        
        if val_metric[1] > best_acc:
            best_acc = val_metric[1]
            best_model_wts = copy.deepcopy(model.state_dict())
        
        if epoch % 20 == 19:
            SaveDict(model.state_dict(), dict_name+'_epoch'+str(epoch+80))
        print()
    
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    print(model.load_state_dict(best_model_wts))
    return best_model_wts, M

In [ ]:
LOSS, M = {}, {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn32_alex = FCN32_Alex()
fcn32_alex_best, M = train_model(fcn32_alex, device, train_loader, val_loader, lr = 1e-3, dict_name = 'fcn32_alex', num_epochs=200)

SaveDict(fcn32_alex_best, 'fcn32_alex_best')
SaveDict(M, 'fcn32_alex_metric')

Epoch 0/199
----------
Batch 0)	3.113	0.043	0.026	0.025	0.041
Batch 1)	2.740	0.224	0.143	0.131	0.199
Batch 2)	1.788	0.797	0.454	0.360	0.609
Batch 3)	1.256	0.813	0.437	0.344	0.633
Batch 4)	1.811	0.682	0.406	0.291	0.495
Batch 5)	1.665	0.753	0.425	0.306	0.526
Batch 6)	1.692	0.774	0.438	0.318	0.554
Batch 7)	1.874	0.731	0.463	0.327	0.535
Batch 8)	1.281	0.759	0.418	0.313	0.577
Batch 9)	1.474	0.705	0.408	0.280	0.509
Batch 10)	1.268	0.733	0.478	0.336	0.574
Batch 11)	1.289	0.766	0.470	0.348	0.566
Batch 12)	1.400	0.748	0.443	0.324	0.534
Batch 13)	1.361	0.751	0.446	0.348	0.595
Batch 14)	1.202	0.758	0.449	0.336	0.559
Batch 15)	1.242	0.746	0.458	0.333	0.558
Batch 16)	1.388	0.676	0.424	0.278	0.440
Batch 17)	1.612	0.671	0.434	0.280	0.453
Batch 18)	1.207	0.755	0.410	0.293	0.555
Batch 19)	1.460	0.718	0.437	0.302	0.532
Batch 20)	1.346	0.729	0.395	0.287	0.539
Batch 21)	1.068	0.775	0.448	0.317	0.558
Batch 22)	1.636	0.641	0.425	0.272	0.435
Batch 23)	1.321	0.720	0.419	0.298	0.521
Batch 24)	1.004	0.793	0.440

Val 199)	0.739	0.782	0.803	0.625	0.604
Val 299)	0.399	0.881	0.571	0.500	0.782
Val 399)	0.686	0.739	0.411	0.386	0.672
Val 499)	0.784	0.800	0.324	0.249	0.546
Val 599)	0.788	0.801	0.606	0.340	0.401
Val 699)	0.419	0.887	0.540	0.483	0.786
Val 799)	0.686	0.785	0.704	0.605	0.641
Val 899)	1.377	0.557	0.455	0.302	0.336
Val 999)	0.187	0.956	0.786	0.730	0.875
Val 1099)	0.455	0.880	0.185	0.172	0.758
Val metric:	[0.6951328035641854, 0.808943944565128, 0.5583045495894667, 0.45140223616399305, 0.6546601484966095]

Epoch 3/199
----------
Batch 0)	0.952	0.736	0.552	0.423	0.583
Batch 1)	0.736	0.788	0.526	0.422	0.616
Batch 2)	0.506	0.875	0.538	0.462	0.738
Batch 3)	0.586	0.834	0.611	0.507	0.704
Batch 4)	0.616	0.827	0.504	0.408	0.672
Batch 5)	0.725	0.812	0.542	0.445	0.650
Batch 6)	0.768	0.782	0.544	0.433	0.611
Batch 7)	0.685	0.820	0.481	0.394	0.670
Batch 8)	0.991	0.731	0.524	0.389	0.520
Batch 9)	0.552	0.857	0.524	0.442	0.704
Batch 10)	0.543	0.848	0.506	0.429	0.705
Batch 11)	0.713	0.772	0.513	0.418	0.606
Ba

Batch 47)	0.578	0.844	0.559	0.466	0.685
Batch 48)	0.534	0.833	0.566	0.470	0.700
Batch 49)	0.761	0.773	0.541	0.414	0.589
Batch 50)	0.375	0.894	0.683	0.582	0.773
Batch 51)	0.474	0.865	0.644	0.545	0.745
Batch 52)	0.660	0.808	0.588	0.480	0.652
Batch 53)	0.572	0.842	0.587	0.482	0.695
Batch 54)	0.613	0.831	0.616	0.507	0.651
Batch 55)	0.430	0.470	0.349	0.268	0.360
Train:	[0.6059251484467829, 0.8213295670147195, 0.5870767125106369, 0.4813524647283481, 0.6700727280314819]
Val 99)	0.600	0.803	0.761	0.601	0.583
Val 199)	0.232	0.916	0.398	0.298	0.742
Val 299)	0.130	0.952	0.652	0.614	0.868
Val 399)	0.162	0.959	0.500	0.472	0.892
Val 499)	0.449	0.879	0.469	0.433	0.798
Val 599)	1.254	0.508	0.541	0.317	0.284
Val 699)	0.147	0.956	0.669	0.615	0.883
Val 799)	0.157	0.964	0.500	0.475	0.904
Val 899)	0.198	0.940	0.567	0.475	0.858
Val 999)	0.263	0.892	0.598	0.527	0.746
Val 1099)	0.360	0.909	0.200	0.169	0.713
Val metric:	[0.5341527391456463, 0.8430603596961707, 0.6238213634419925, 0.5138758254311249, 0.69419097

Batch 35)	0.429	0.867	0.667	0.577	0.753
Batch 36)	0.528	0.834	0.567	0.458	0.680
Batch 37)	0.452	0.865	0.603	0.510	0.722
Batch 38)	0.442	0.861	0.612	0.517	0.725
Batch 39)	0.420	0.861	0.640	0.552	0.722
Batch 40)	0.693	0.812	0.526	0.440	0.695
Batch 41)	0.573	0.807	0.592	0.491	0.635
Batch 42)	0.540	0.842	0.622	0.527	0.690
Batch 43)	0.410	0.862	0.651	0.542	0.712
Batch 44)	0.630	0.798	0.618	0.498	0.651
Batch 45)	0.525	0.836	0.712	0.585	0.679
Batch 46)	0.564	0.828	0.594	0.490	0.692
Batch 47)	0.676	0.782	0.602	0.487	0.626
Batch 48)	0.679	0.777	0.604	0.472	0.634
Batch 49)	0.474	0.853	0.589	0.500	0.730
Batch 50)	0.600	0.829	0.570	0.487	0.687
Batch 51)	0.465	0.872	0.632	0.516	0.726
Batch 52)	0.614	0.813	0.605	0.479	0.650
Batch 53)	0.559	0.822	0.594	0.492	0.667
Batch 54)	0.462	0.865	0.685	0.581	0.737
Batch 55)	0.341	0.501	0.394	0.346	0.427
Train:	[0.5257279447299971, 0.8397413123285639, 0.6201425393105492, 0.5134706756823745, 0.6920653942800036]
Val 99)	0.066	0.996	0.500	0.495	0.980
Val 199)	0.184

Batch 22)	0.471	0.849	0.654	0.545	0.700
Batch 23)	0.416	0.862	0.695	0.591	0.718
Batch 24)	0.448	0.862	0.662	0.560	0.738
Batch 25)	0.323	0.907	0.690	0.605	0.781
Batch 26)	0.629	0.803	0.573	0.448	0.629
Batch 27)	0.448	0.855	0.634	0.520	0.712
Batch 28)	0.528	0.833	0.623	0.515	0.681
Batch 29)	0.488	0.851	0.663	0.556	0.706
Batch 30)	0.428	0.879	0.671	0.577	0.754
Batch 31)	0.407	0.877	0.649	0.568	0.751
Batch 32)	0.542	0.828	0.607	0.509	0.688
Batch 33)	0.473	0.855	0.666	0.565	0.708
Batch 34)	0.381	0.885	0.671	0.575	0.718
Batch 35)	0.505	0.842	0.570	0.473	0.692
Batch 36)	0.368	0.887	0.639	0.550	0.748
Batch 37)	0.426	0.874	0.712	0.541	0.650
Batch 38)	0.575	0.820	0.642	0.510	0.683
Batch 39)	0.476	0.847	0.623	0.501	0.695
Batch 40)	0.457	0.859	0.627	0.534	0.704
Batch 41)	0.524	0.832	0.682	0.567	0.707
Batch 42)	0.501	0.829	0.677	0.561	0.703
Batch 43)	0.397	0.878	0.608	0.497	0.739
Batch 44)	0.458	0.855	0.664	0.553	0.681
Batch 45)	0.597	0.816	0.604	0.491	0.643
Batch 46)	0.589	0.819	0.653	0.530	0.666


...

Val 99)	0.187	0.937	0.933	0.707	0.783
Val 199)	0.302	0.902	0.847	0.767	0.798
Val 299)	0.070	0.983	0.875	0.769	0.945
Val 399)	0.280	0.937	0.497	0.346	0.145
Val 499)	0.062	0.979	0.820	0.733	0.935
Val 599)	0.152	0.945	0.908	0.807	0.848
Val 699)	0.104	0.965	0.728	0.632	0.886
Val 799)	0.063	0.978	0.977	0.918	0.896
Val 899)	0.270	0.898	0.896	0.761	0.735
Val 999)	0.144	0.966	0.973	0.909	0.907
Val 1099)	0.145	0.967	0.957	0.910	0.893
Val metric:	[0.16037677312110535, 0.9476008739047165, 0.8254253351668642, 0.7152865729607785, 0.8255662686229338]

Epoch 85/199
----------
Batch 0)	0.152	0.949	0.765	0.681	0.863
Batch 1)	0.202	0.931	0.780	0.664	0.779
Batch 2)	0.246	0.911	0.791	0.671	0.774
Batch 3)	0.169	0.944	0.838	0.728	0.806
Batch 4)	0.155	0.949	0.841	0.739	0.860
Batch 5)	0.144	0.952	0.725	0.639	0.851
Batch 6)	0.194	0.936	0.759	0.659	0.812
Batch 7)	0.182	0.941	0.804	0.709	0.840
Batch 8)	0.228	0.924	0.806	0.702	0.787
Batch 9)	0.220	0.924	0.771	0.671	0.785
Batch 10)	0.205	0.931	0.763	0.655	0.783
Ba

Batch 46)	0.206	0.928	0.788	0.696	0.810
Batch 47)	0.208	0.930	0.748	0.656	0.798
Batch 48)	0.213	0.924	0.820	0.709	0.793
Batch 49)	0.171	0.944	0.794	0.695	0.836
Batch 50)	0.207	0.925	0.805	0.680	0.769
Batch 51)	0.187	0.937	0.852	0.719	0.787
Batch 52)	0.212	0.925	0.783	0.683	0.798
Batch 53)	0.177	0.943	0.744	0.660	0.836
Batch 54)	0.202	0.932	0.820	0.699	0.777
Batch 55)	0.089	0.572	0.491	0.448	0.520
Train:	[0.1884233092607783, 0.9351312746347008, 0.7907820472050343, 0.6880914761607022, 0.8078332041711416]
Val 99)	0.175	0.905	0.567	0.457	0.688
Val 199)	0.095	0.977	0.969	0.922	0.928
Val 299)	0.151	0.946	0.859	0.732	0.894
Val 399)	0.183	0.942	0.874	0.749	0.843
Val 499)	0.065	0.980	0.982	0.905	0.911
Val 599)	0.588	0.795	0.783	0.590	0.517
Val 699)	0.181	0.956	0.931	0.818	0.848
Val 799)	0.087	0.969	0.952	0.849	0.898
Val 899)	0.171	0.937	0.934	0.858	0.833
Val 999)	0.151	0.954	0.917	0.813	0.871
Val 1099)	0.202	0.952	0.947	0.868	0.866
Val metric:	[0.16049296343590122, 0.9477749650329935, 0.8308884

Batch 34)	0.209	0.927	0.770	0.652	0.797
Batch 35)	0.224	0.924	0.807	0.687	0.759
Batch 36)	0.182	0.941	0.759	0.660	0.828
Batch 37)	0.180	0.938	0.817	0.719	0.832
Batch 38)	0.179	0.939	0.880	0.769	0.820
Batch 39)	0.187	0.936	0.752	0.653	0.808
Batch 40)	0.198	0.936	0.742	0.641	0.801
Batch 41)	0.137	0.955	0.806	0.710	0.838
Batch 42)	0.168	0.939	0.779	0.691	0.837
Batch 43)	0.175	0.940	0.831	0.725	0.817
Batch 44)	0.191	0.936	0.767	0.680	0.811
Batch 45)	0.232	0.919	0.760	0.658	0.784
Batch 46)	0.171	0.939	0.833	0.727	0.814
Batch 47)	0.164	0.946	0.856	0.747	0.811
Batch 48)	0.156	0.947	0.807	0.682	0.802
Batch 49)	0.183	0.939	0.824	0.739	0.822
Batch 50)	0.197	0.929	0.771	0.670	0.811
Batch 51)	0.187	0.935	0.791	0.706	0.824
Batch 52)	0.194	0.934	0.836	0.702	0.746
Batch 53)	0.178	0.940	0.767	0.665	0.838
Batch 54)	0.223	0.921	0.758	0.640	0.774
Batch 55)	0.136	0.548	0.497	0.416	0.450
Train:	[0.18459615143511793, 0.9366860751604205, 0.7930231896581401, 0.6912834743609854, 0.8098213525330413]
Val 99)	0.1

Batch 21)	0.192	0.930	0.807	0.693	0.797
Batch 22)	0.175	0.941	0.839	0.725	0.799
Batch 23)	0.161	0.944	0.780	0.678	0.818
Batch 24)	0.154	0.945	0.831	0.713	0.782
Batch 25)	0.198	0.931	0.834	0.732	0.810
Batch 26)	0.214	0.928	0.762	0.662	0.788
Batch 27)	0.184	0.933	0.870	0.734	0.772
Batch 28)	0.168	0.940	0.880	0.771	0.822
Batch 29)	0.215	0.926	0.819	0.715	0.797
Batch 30)	0.193	0.933	0.796	0.703	0.805
Batch 31)	0.185	0.936	0.792	0.670	0.783
Batch 32)	0.170	0.942	0.837	0.718	0.780
Batch 33)	0.151	0.950	0.798	0.690	0.805
Batch 34)	0.181	0.944	0.743	0.655	0.812
Batch 35)	0.157	0.946	0.766	0.678	0.843
Batch 36)	0.187	0.935	0.790	0.681	0.791
Batch 37)	0.246	0.915	0.738	0.635	0.776
Batch 38)	0.187	0.936	0.771	0.659	0.791
Batch 39)	0.188	0.935	0.821	0.719	0.798
Batch 40)	0.171	0.939	0.798	0.702	0.807
Batch 41)	0.192	0.938	0.819	0.709	0.770
Batch 42)	0.168	0.942	0.750	0.669	0.830
Batch 43)	0.192	0.936	0.721	0.625	0.803
Batch 44)	0.148	0.949	0.839	0.721	0.843
Batch 45)	0.183	0.936	0.806	0.717	0.833


Batch 8)	0.134	0.954	0.847	0.736	0.828
Batch 9)	0.188	0.937	0.797	0.715	0.825
Batch 10)	0.179	0.940	0.773	0.692	0.825
Batch 11)	0.198	0.926	0.792	0.689	0.799
Batch 12)	0.165	0.940	0.813	0.717	0.815
Batch 13)	0.187	0.937	0.762	0.672	0.820
Batch 14)	0.200	0.928	0.749	0.639	0.816
Batch 15)	0.153	0.946	0.791	0.691	0.837
Batch 16)	0.187	0.937	0.805	0.700	0.772
Batch 17)	0.173	0.944	0.844	0.746	0.844
Batch 18)	0.169	0.942	0.777	0.676	0.796
Batch 19)	0.168	0.945	0.842	0.743	0.832
Batch 20)	0.180	0.933	0.838	0.714	0.760
Batch 21)	0.164	0.942	0.735	0.662	0.833
Batch 22)	0.151	0.948	0.794	0.709	0.831
Batch 23)	0.153	0.946	0.794	0.715	0.842
Batch 24)	0.192	0.931	0.813	0.694	0.797
Batch 25)	0.136	0.955	0.803	0.709	0.861
Batch 26)	0.172	0.941	0.818	0.691	0.794
Batch 27)	0.164	0.948	0.767	0.679	0.836
Batch 28)	0.183	0.939	0.775	0.668	0.792
Batch 29)	0.216	0.925	0.824	0.685	0.731
Batch 30)	0.151	0.947	0.784	0.708	0.869
Batch 31)	0.155	0.945	0.757	0.671	0.838
Batch 32)	0.175	0.941	0.806	0.714	0.832
Ba

In [11]:
# continue after 80 epoch because of the power loss 

LOSS, M = {}, {}
device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn32_alex = FCN32_Alex()
fcn32dict = LoadDict('fcn32_alex_epoch79')
fcn32_alex.load_state_dict(fcn32dict, strict = True)

fcn32_alex_best, M = train_model(fcn32_alex, device, train_loader, val_loader, lr = 1e-3, dict_name = 'fcn32_alex', num_epochs=120)

SaveDict(fcn32_alex_best, 'fcn32_alex_best')
SaveDict(M, 'fcn32_alex_metric')

Epoch 0/119
----------
Batch 0)	0.165	0.943	0.756	0.652	0.835
Batch 1)	0.241	0.915	0.785	0.685	0.783
Batch 2)	0.194	0.935	0.798	0.687	0.828
Batch 3)	0.204	0.928	0.758	0.656	0.818
Batch 4)	0.239	0.918	0.760	0.649	0.769
Batch 5)	0.174	0.941	0.781	0.679	0.833
Batch 6)	0.186	0.942	0.802	0.660	0.752
Batch 7)	0.179	0.938	0.858	0.749	0.825
Batch 8)	0.177	0.941	0.818	0.708	0.791
Batch 9)	0.176	0.937	0.816	0.716	0.792
Batch 10)	0.145	0.952	0.787	0.702	0.856
Batch 11)	0.229	0.921	0.748	0.643	0.778
Batch 12)	0.189	0.932	0.784	0.685	0.810
Batch 13)	0.213	0.928	0.773	0.677	0.789
Batch 14)	0.194	0.927	0.794	0.693	0.816
Batch 15)	0.149	0.949	0.841	0.748	0.854
Batch 16)	0.150	0.945	0.847	0.750	0.846
Batch 17)	0.166	0.943	0.872	0.765	0.822
Batch 18)	0.201	0.931	0.839	0.720	0.771
Batch 19)	0.165	0.945	0.762	0.681	0.840
Batch 20)	0.155	0.950	0.762	0.680	0.838
Batch 21)	0.200	0.931	0.802	0.685	0.788
Batch 22)	0.197	0.931	0.733	0.644	0.806
Batch 23)	0.188	0.942	0.794	0.670	0.769
Batch 24)	0.207	0.932	0.764

Val 199)	0.094	0.965	0.891	0.726	0.909
Val 299)	0.186	0.941	0.942	0.844	0.812
Val 399)	0.140	0.955	0.954	0.890	0.868
Val 499)	0.300	0.911	0.829	0.668	0.746
Val 599)	0.046	0.989	0.842	0.732	0.946
Val 699)	0.493	0.887	0.862	0.696	0.609
Val 799)	0.191	0.926	0.929	0.801	0.749
Val 899)	0.184	0.937	0.548	0.439	0.827
Val 999)	0.094	0.973	0.912	0.840	0.913
Val 1099)	0.116	0.961	0.753	0.644	0.894
Val metric:	[0.16417702071747595, 0.9465724360243251, 0.8140379798093101, 0.711303796311842, 0.8232354617121586]

Epoch 3/119
----------
Batch 0)	0.195	0.934	0.814	0.729	0.819
Batch 1)	0.169	0.943	0.812	0.701	0.789
Batch 2)	0.180	0.938	0.768	0.678	0.824
Batch 3)	0.215	0.924	0.842	0.735	0.779
Batch 4)	0.196	0.935	0.821	0.712	0.783
Batch 5)	0.185	0.932	0.805	0.688	0.778
Batch 6)	0.212	0.928	0.770	0.676	0.817
Batch 7)	0.178	0.941	0.857	0.759	0.847
Batch 8)	0.207	0.930	0.755	0.652	0.796
Batch 9)	0.168	0.944	0.822	0.710	0.837
Batch 10)	0.230	0.918	0.789	0.691	0.800
Batch 11)	0.202	0.926	0.751	0.663	0.805
Ba

Batch 47)	0.143	0.951	0.795	0.705	0.853
Batch 48)	0.186	0.932	0.862	0.742	0.812
Batch 49)	0.192	0.934	0.851	0.757	0.791
Batch 50)	0.174	0.943	0.724	0.641	0.841
Batch 51)	0.206	0.931	0.777	0.678	0.812
Batch 52)	0.193	0.934	0.807	0.709	0.827
Batch 53)	0.188	0.931	0.796	0.680	0.764
Batch 54)	0.182	0.931	0.797	0.686	0.800
Batch 55)	0.135	0.550	0.506	0.418	0.439
Train:	[0.18999150357872463, 0.9346829206371012, 0.7886921086241784, 0.6869760672334999, 0.8073087475825876]
Val 99)	0.039	0.989	0.970	0.866	0.951
Val 199)	0.089	0.960	0.950	0.784	0.907
Val 299)	0.044	0.993	0.930	0.883	0.966
Val 399)	0.255	0.923	0.868	0.756	0.813
Val 499)	0.172	0.955	0.938	0.848	0.836
Val 599)	0.156	0.962	0.654	0.584	0.853
Val 699)	0.090	0.974	0.919	0.857	0.927
Val 799)	0.320	0.908	0.454	0.358	0.692
Val 899)	0.150	0.950	0.694	0.626	0.859
Val 999)	0.224	0.917	0.913	0.810	0.770
Val 1099)	0.072	0.979	0.973	0.853	0.899
Val metric:	[0.15917379292470848, 0.9483201098731338, 0.8252198995036097, 0.7160483218612843, 0.825999

Batch 35)	0.160	0.946	0.792	0.697	0.816
Batch 36)	0.170	0.939	0.781	0.701	0.827
Batch 37)	0.174	0.945	0.666	0.579	0.800
Batch 38)	0.139	0.949	0.816	0.732	0.856
Batch 39)	0.210	0.931	0.719	0.603	0.751
Batch 40)	0.160	0.944	0.802	0.711	0.817
Batch 41)	0.224	0.919	0.812	0.711	0.787
Batch 42)	0.166	0.940	0.753	0.658	0.813
Batch 43)	0.207	0.928	0.777	0.650	0.768
Batch 44)	0.142	0.954	0.854	0.753	0.818
Batch 45)	0.169	0.945	0.819	0.728	0.838
Batch 46)	0.233	0.918	0.747	0.648	0.800
Batch 47)	0.204	0.931	0.811	0.699	0.797
Batch 48)	0.146	0.951	0.832	0.734	0.853
Batch 49)	0.179	0.941	0.782	0.701	0.826
Batch 50)	0.173	0.941	0.792	0.704	0.832
Batch 51)	0.226	0.919	0.779	0.670	0.794
Batch 52)	0.207	0.927	0.799	0.705	0.807
Batch 53)	0.189	0.933	0.834	0.731	0.809
Batch 54)	0.171	0.943	0.727	0.636	0.820
Batch 55)	0.130	0.553	0.526	0.438	0.458
Train:	[0.18979045662948554, 0.9348190064370533, 0.7888862825980003, 0.687300501328572, 0.807395035631066]
Val 99)	0.128	0.960	0.928	0.829	0.846
Val 199)	0.162	

Batch 22)	0.218	0.921	0.805	0.692	0.795
Batch 23)	0.198	0.933	0.758	0.662	0.815
Batch 24)	0.172	0.941	0.791	0.697	0.824
Batch 25)	0.166	0.941	0.817	0.728	0.830
Batch 26)	0.268	0.907	0.743	0.643	0.729
Batch 27)	0.198	0.933	0.754	0.662	0.770
Batch 28)	0.146	0.953	0.860	0.726	0.756
Batch 29)	0.168	0.938	0.810	0.717	0.837
Batch 30)	0.179	0.937	0.783	0.679	0.812
Batch 31)	0.197	0.934	0.822	0.724	0.790
Batch 32)	0.159	0.947	0.824	0.723	0.822
Batch 33)	0.206	0.929	0.775	0.678	0.805
Batch 34)	0.183	0.937	0.806	0.683	0.767
Batch 35)	0.218	0.917	0.733	0.628	0.776
Batch 36)	0.203	0.925	0.787	0.679	0.777
Batch 37)	0.190	0.936	0.835	0.736	0.813
Batch 38)	0.226	0.926	0.805	0.699	0.791
Batch 39)	0.170	0.943	0.842	0.727	0.821
Batch 40)	0.174	0.943	0.820	0.721	0.837
Batch 41)	0.182	0.943	0.710	0.625	0.833
Batch 42)	0.188	0.937	0.829	0.702	0.793
Batch 43)	0.178	0.937	0.804	0.691	0.791
Batch 44)	0.194	0.931	0.794	0.693	0.821
Batch 45)	0.172	0.939	0.840	0.751	0.827
Batch 46)	0.144	0.953	0.830	0.744	0.845


...

Batch 16)	0.127	0.957	0.798	0.700	0.841
Batch 17)	0.097	0.968	0.857	0.768	0.873
Batch 18)	0.144	0.949	0.869	0.765	0.827
Batch 19)	0.155	0.946	0.776	0.684	0.813
Batch 20)	0.103	0.964	0.795	0.725	0.877
Batch 21)	0.135	0.951	0.829	0.751	0.848
Batch 22)	0.178	0.938	0.775	0.661	0.791
Batch 23)	0.144	0.953	0.857	0.760	0.836
Batch 24)	0.140	0.951	0.842	0.745	0.846
Batch 25)	0.134	0.949	0.790	0.679	0.833
Batch 26)	0.137	0.953	0.838	0.735	0.828
Batch 27)	0.133	0.954	0.860	0.761	0.838
Batch 28)	0.109	0.962	0.790	0.684	0.827
Batch 29)	0.144	0.948	0.831	0.739	0.832
Batch 30)	0.122	0.958	0.834	0.760	0.870
Batch 31)	0.132	0.951	0.799	0.708	0.801
Batch 32)	0.150	0.948	0.845	0.757	0.839
Batch 33)	0.152	0.945	0.858	0.755	0.812
Batch 34)	0.111	0.961	0.836	0.703	0.849
Batch 35)	0.149	0.950	0.797	0.692	0.829
Batch 36)	0.118	0.958	0.861	0.765	0.849
Batch 37)	0.124	0.953	0.881	0.789	0.849
Batch 38)	0.132	0.953	0.833	0.702	0.789
Batch 39)	0.139	0.948	0.840	0.742	0.838
Batch 40)	0.112	0.961	0.887	0.794	0.836


Batch 3)	0.170	0.940	0.858	0.753	0.794
Batch 4)	0.132	0.954	0.783	0.690	0.849
Batch 5)	0.132	0.953	0.807	0.705	0.850
Batch 6)	0.126	0.957	0.815	0.732	0.847
Batch 7)	0.147	0.948	0.869	0.778	0.840
Batch 8)	0.173	0.936	0.824	0.723	0.802
Batch 9)	0.157	0.944	0.815	0.716	0.831
Batch 10)	0.138	0.950	0.859	0.752	0.826
Batch 11)	0.139	0.951	0.852	0.757	0.828
Batch 12)	0.095	0.965	0.840	0.736	0.843
Batch 13)	0.124	0.954	0.843	0.761	0.853
Batch 14)	0.131	0.953	0.833	0.735	0.835
Batch 15)	0.115	0.964	0.867	0.772	0.854
Batch 16)	0.122	0.957	0.854	0.763	0.846
Batch 17)	0.113	0.960	0.845	0.757	0.864
Batch 18)	0.198	0.930	0.784	0.652	0.759
Batch 19)	0.106	0.963	0.820	0.704	0.833
Batch 20)	0.123	0.959	0.888	0.753	0.768
Batch 21)	0.131	0.954	0.865	0.775	0.840
Batch 22)	0.137	0.950	0.746	0.640	0.805
Batch 23)	0.135	0.954	0.863	0.753	0.830
Batch 24)	0.138	0.953	0.825	0.725	0.848
Batch 25)	0.114	0.959	0.808	0.707	0.846
Batch 26)	0.135	0.953	0.815	0.719	0.851
Batch 27)	0.146	0.949	0.826	0.727	0.838
Batch 2

Val 599)	0.117	0.962	0.970	0.888	0.890
Val 699)	0.057	0.979	0.950	0.716	0.890
Val 799)	0.057	0.981	0.979	0.907	0.854
Val 899)	0.167	0.946	0.707	0.579	0.771
Val 999)	0.215	0.917	0.778	0.563	0.723
Val 1099)	0.070	0.978	0.976	0.908	0.917
Val metric:	[0.11623999285893087, 0.9611377685228549, 0.8743329103791625, 0.7561640961934828, 0.8463345221902784]

Epoch 116/119
----------
Batch 0)	0.124	0.957	0.853	0.769	0.858
Batch 1)	0.141	0.952	0.777	0.695	0.848
Batch 2)	0.128	0.956	0.811	0.722	0.854
Batch 3)	0.134	0.953	0.799	0.711	0.830
Batch 4)	0.149	0.945	0.833	0.743	0.837
Batch 5)	0.112	0.961	0.822	0.724	0.858
Batch 6)	0.111	0.962	0.869	0.776	0.847
Batch 7)	0.173	0.939	0.798	0.695	0.816
Batch 8)	0.137	0.953	0.845	0.760	0.842
Batch 9)	0.126	0.957	0.845	0.740	0.816
Batch 10)	0.124	0.957	0.865	0.767	0.839
Batch 11)	0.181	0.937	0.757	0.649	0.768
Batch 12)	0.132	0.952	0.823	0.722	0.831
Batch 13)	0.108	0.965	0.795	0.705	0.872
Batch 14)	0.139	0.950	0.840	0.719	0.803
Batch 15)	0.116	0.960	0.857	0.763	0

Batch 51)	0.165	0.947	0.792	0.669	0.766
Batch 52)	0.118	0.958	0.881	0.788	0.858
Batch 53)	0.121	0.960	0.818	0.743	0.863
Batch 54)	0.118	0.961	0.807	0.720	0.856
Batch 55)	0.075	0.574	0.552	0.498	0.505
Train:	[0.1316664966908719, 0.9537138255711087, 0.8362651675363605, 0.7356864131444649, 0.8341006374867328]
Val 99)	0.046	0.986	0.989	0.903	0.944
Val 199)	0.049	0.987	0.987	0.895	0.821
Val 299)	0.082	0.969	0.894	0.739	0.868
Val 399)	0.087	0.973	0.944	0.818	0.876
Val 499)	0.030	0.987	0.971	0.781	0.956
Val 599)	0.048	0.984	0.965	0.513	0.366
Val 699)	0.293	0.893	0.853	0.742	0.700
Val 799)	0.021	0.991	0.965	0.810	0.967
Val 899)	0.040	0.992	0.940	0.822	0.954
Val 999)	0.044	0.987	0.974	0.866	0.937
Val 1099)	0.081	0.982	0.916	0.837	0.921
Val metric:	[0.11017913048630908, 0.9633362614668993, 0.8697077224718935, 0.7603301086940846, 0.8489144532728019]

Epoch 119/119
----------
Batch 0)	0.121	0.959	0.787	0.673	0.801
Batch 1)	0.177	0.937	0.808	0.696	0.807
Batch 2)	0.108	0.965	0.824	0.741	0.868
Batch 

0